In [63]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import pandas as pd

In [2]:
# 테스트 브라우저 열어보기
# https://map.naver.com/p?c=15.00,0,0,0,dh
driver = wb.Chrome()
url = "https://map.naver.com/p?c=15.00,0,0,0,dh"
driver.get(url)

In [ ]:
# 이수역 맛집
search = driver.find_element(By.ID, "input_search1763950216170")

In [ ]:
search.send_keys("이수역 맛집")
search.send_keys(Keys.ENTER)

In [3]:
# 이수역 맛집 검색 -> 새로운 방법
query = input("검색할 지역의 맛집>> ")

검색할 지역의 맛집>>  이수역 맛집


In [4]:
# 검색란 가져오기 -> class로
search_input = driver.find_element(By.CSS_SELECTOR, ".input_search")

In [5]:
search_input.send_keys(query + '\n')

In [6]:
# 가게명 수집하기
driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li:nth-child(2) > div.CHC5F > div.bSoi3 > a > span.TYaxT")

[]

In [7]:
driver2 = wb.Chrome()
url2 = 'https://pcmap.place.naver.com/place/list?query=%ED%99%8D%EB%8C%80%20%EB%A7%9B%EC%A7%91&x=126.98067960000259&y=37.4863012999992&clientX=127.035639&clientY=37.641403&bounds=126.97653826928484%3B37.480043748316305%3B126.98503550744374%3B37.49238807526318&display=70&ts=1763951075230&additionalHeight=76&locale=ko&mapUrl=https%3A%2F%2Fmap.naver.com%2Fp%2Fsearch%2F%ED%99%8D%EB%8C%80%20%EB%A7%9B%EC%A7%91'
driver2.get(url2)

In [8]:
food_store = driver2.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > div.bSoi3 > a > span.TYaxT")

In [9]:
food_store[0].text

'디어리스트 연남'

In [10]:
# iframe 창으로 전환하는 함수
driver.switch_to.frame("searchIframe") # iframe의 id값만 가져오기

In [11]:
# 가게명 수집하기
food_stores = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li > div.CHC5F > div.bSoi3 > a > span.TYaxT")

In [12]:
food_stores[0].text

'이곱착한고기 사당본점'

## 가게명 이외에 가게 주소, 전화번호 수집

In [13]:
# 가게명 클릭하기 -> 가게명 클릭했을 때 상세페이지 내에 주소, 전화번호가 있기 때문
food_stores[0].click()

In [14]:
# 이전 창으로 되돌아가는 함수 -> 메인으로 접근(iframe에서 나간다)
driver.switch_to.default_content()

In [15]:
# 상세페이지 내 iframe으로 다시 접근
driver.switch_to.frame("entryIframe")

In [18]:
# 상세페이지 내 주소 가져오기
store_add = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH")

In [19]:
store_add[0].text

'서울 동작구 동작대로3길 6'

In [20]:
# 상세페이지 내 전화번호 가져오기
store_number = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div:nth-child(6) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q")

In [21]:
store_number[0].text

'0507-1359-8298'

In [22]:
# 이전페이지로 이동
driver.switch_to.default_content()

In [23]:
# iframe으로 이동
driver.switch_to.frame("searchIframe")

In [24]:
driver.quit()

### 실습 -> 가게정보 수집하기(1page 내 스크롤 포함)
- 스크롤 정보를 가지고 있는 iframe 내 요소: #_pcmap_list_scroll_container

In [55]:
# 1. 브라우저 열기
driver = wb.Chrome()
url = "https://map.naver.com/p?c=15.00,0,0,0,dh"
driver.get(url)

In [56]:
# 2. 요청
#query = input("검색할 곳 입력 >> ")
query = "홍대 맛집"
# 3. 검색
search_input = driver.find_element(By.CSS_SELECTOR, ".input_search")
search_input.send_keys(query + "\n")

In [57]:
# 4. 창전환
driver.switch_to.frame("searchIframe")

In [58]:
# 5. 스크롤 하강 (#_pcmap_list_scroll_container)
while True:
    old = driver.page_source
    scroll = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
    scroll.send_keys(Keys.END)
    driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container").send_keys(Keys.END)
    time.sleep(2)
    new = driver.page_source

    if old == new:
        print("페이지 하강 완료")
        break
    else:
        pass

페이지 하강 완료


In [59]:
# 6. 데이터 수집 -> 가게명(store_names), 주소(store_adds), 전화번호(store_numbers)를 각각의 리스트에 담기
# 6-1. 전체 데이터 가져와야하기 때문에 요소 먼저 가져오기 -> 전체 가져와야 하니까 li 뒤 단수를 표현하는 nth-child 지워주기
food_stores = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li > div.CHC5F > div.bSoi3 > a > span.TYaxT")
# 6-2. 반복하여 수집하기 -> 스크롤을 통해 전체 요소를 가져왔기 때문에 몇 개를 수집해야하는지 확인 가능(for)
store_names = []
store_adds = []
store_numbers = []
for i in range(len(food_stores)):
    # 6-3. 클릭할 요소 가져오기
    food_stores = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li > div.CHC5F > div.bSoi3 > a > span.TYaxT")
    food_stores[i].click()
    time.sleep(1)

    # 6-4. 다시 상세 페이지의 데이터를 보기 위한 창전환
    driver.switch_to.default_content()
    driver.switch_to.frame("entryIframe")

    # 6-5. 상세페이지 내 가게명 가져오기
    # 가게명의 선택지 : #_title > div > span.GHAhO
    try:
        store_name = driver.find_element(By.CSS_SELECTOR, "#_title > div > span.GHAhO").text
        store_names.append(store_name)
    except:
        store_names.append("오류발생")
    # 주소
    try:
        store_add = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div > div > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH").text
        store_adds.append(store_add)
    except:
        store_adds.append("없음")
    # 전화번호
    try:
        store_number = driver.find_element(By.CSS_SELECTOR, "#app-root > div > div > div > div > div > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q").text
        store_numbers.append(store_number)
    except:
        store_numbers.append("없음")
        
    driver.switch_to.default_content() # 창전환
    driver.switch_to.frame("searchIframe") # 가게명 클릭하러 가기 위한(상세페이지 접근 위한) 프레임으로 이동
# 종료
driver.quit()

In [60]:
store_names

['솥돈 홍대점',
 '디어리스트 연남',
 '바다회사랑 서교점',
 '대나무숙성회관 홍대점',
 '키친485 홍대',
 '연남족발1987 홍대점',
 '구사 연남',
 '아벨롭 홍대본점',
 '합정광안리 숙성횟집 2호점',
 '바다생고기',
 '더기와 합정점',
 '합정광안리 숙성횟집 본점',
 '부야스곱창 홍대본점',
 '육산장 홍대본점',
 '장인닭갈비 홍대2호점',
 '조연탄 합정점',
 '오밀 홍대',
 '동주 합정점',
 '셔우드',
 '카레시 합정본점',
 '더플레이스 홍대L7점',
 '단막 합정점',
 '비스트로사랑방',
 '조블라 홍대점',
 '무제서울',
 '귀한족발 홍대서교점',
 '몽달 홍대점',
 '두루미 해장집',
 '동백가옥 합정점',
 '라헬의부엌 홍대점',
 '황곱 황소곱창',
 '기린1인용회전훠궈',
 '테이커테이블 연남점',
 '오키도키1호점',
 '하이디라오 홍대점',
 '멘노아지 홍대점',
 '카가와 합정본점',
 '추파 홍대점',
 '오늘은내가쏜다',
 '23플래터',
 '더기와 합정점',
 '옹시미',
 '코노미',
 '카페 플로라운지',
 '스케줄합정',
 '컴피',
 '키친205 홍대점',
 '토모토카레 홍대점',
 '오마카세 오사이초밥 홍대점',
 '집밥김치찌개 본점',
 '피오니',
 '합정공감']

In [61]:
store_adds

['서울 마포구 잔다리로 35 1층 101호',
 '서울 마포구 동교로27길 21-8 1층',
 '서울 마포구 동교로 143',
 '서울 마포구 동교로 212-9 1층',
 '서울 마포구 양화로6길 67 1층',
 '서울 마포구 와우산로29길 69 1층 101호',
 '서울 마포구 동교로30길 6 연트럴타워 지하1층',
 '서울 마포구 양화로15안길 6 Avelop',
 '서울 마포구 독막로3길 50 합정광안리 2호점',
 '서울 마포구 와우산로29나길 7 1층 바다생고기',
 '서울 마포구 독막로3길 25 1층',
 '서울 마포구 독막로3길 38 합정광안리 본점',
 '서울 마포구 어울마당로 136-13 2층',
 '서울 마포구 와우산로21길 31-11 1층',
 '서울 마포구 어울마당로 111-1 2층',
 '서울 마포구 양화로8길 13 지1층',
 '서울 마포구 양화로 178-8 2층',
 '서울 마포구 독막로7길 41',
 '서울 마포구 와우산로37길 11 2층',
 '서울 마포구 독막로9길 31 1층',
 '서울 마포구 양화로 141 L7홍대 21층',
 '서울 마포구 독막로7길 21 1, 2층',
 '서울 마포구 와우산로15길 40 BISTRO 사랑방 , 2층',
 '서울 마포구 어울마당로 136-10 1층',
 '서울 마포구 양화로6길 91 1층',
 '서울 마포구 동교로 127 1층(서교동)',
 '서울 마포구 월드컵북로2길 31 1층',
 '서울 마포구 동교로 147-1 1층 (서교동)',
 '서울 마포구 양화로6길 38 1층 동백가옥 합정점',
 '서울 마포구 와우산로29길 47 1,2층',
 '서울 마포구 어울마당로 149 1층',
 '서울 마포구 홍익로6길 38 2층',
 '서울 마포구 동교로30길 6 1층',
 '서울 마포구 와우산로17길 15 2층',
 '서울 마포구 양화로 176 동교동 와이즈파크 5층',
 '서울 마포구 홍익로 10 지하1층 지141,142호',
 '서울 마포구 잔다리로 66 1층',
 '서울 마포구 홍익로3길 44

In [62]:
store_numbers

['0507-1470-2779',
 '0507-1333-6861',
 '02-325-3435',
 '0507-1495-9049',
 '0507-1406-0485',
 '0507-1384-1988',
 '없음',
 '02-336-8897',
 '0507-1335-7367',
 '02-6052-9098',
 '0507-1416-1913',
 '0507-1314-7366',
 '02-3144-4967',
 '0507-1428-7226',
 '없음',
 '0507-1475-5539',
 '0507-1312-9197',
 '0507-1431-0122',
 '070-7543-6590',
 '0507-1310-6363',
 '0507-1373-1372',
 '0507-1326-4531',
 '070-4799-0368',
 '0507-1355-9243',
 '0507-1326-7630',
 '0507-1307-3225',
 '0507-1367-9712',
 '0507-1383-6605',
 '0507-1420-9685',
 '0507-1464-4325',
 '0507-1492-8310',
 '02-6081-9319',
 '0507-1376-0066',
 '0507-1447-1804',
 '0507-1381-7668',
 '0507-1487-1227',
 '0507-1421-1434',
 '0507-1303-5582',
 '0507-1344-7762',
 '0507-1340-5417',
 '0507-1416-1913',
 '0507-1407-2399',
 '02-333-3036',
 '0507-1416-6547',
 '0507-1338-2622',
 '0507-1330-4489',
 '02-336-8273',
 '0507-1403-4200',
 '0507-1348-5425',
 '02-332-2866',
 '02-333-5325',
 '02-333-4733']

In [64]:
food_df = pd.DataFrame({"가게명":store_names, "가게주소":store_adds, "가게번호":store_numbers})

In [65]:
food_df.to_csv("맛집리스트.csv", encoding = "utf-8-sig")